## 3.1 Scrape tweets from selected politicians

### 1. Import packages

As we want to analyze Twitter data and speeches from German politicians one of our first tasks was to get the tweets. Because we decided to analyze the nineteenth Bundestag as our time span, we had to get data from politicians of the six represented parties in the Bundestag. <br>
For our purpose we decided to take seven of the more active and most followed politicians from each respected political party. After deciding on which politicians accounts need to be scraped, we implemented a scraper. We chose snscrape which is a scraper for social network services. It allowed us to scrape every ever-posted tweet by one of our politicians without problems and restrictions. Apart from Twitter this scraper can be used for many more social networks like Facebook or Instagram but for our project Twitter was sufficient.  

In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
from tqdm.notebook import tqdm

tqdm.pandas()

### 2. Define variables and functions

After installation this package allowed us to scrape tweets from desired accounts and gave us many options on how the retrieved data should look. Besides the accounts username and tweet texts it allowed us to also get information on the time of the post, the displayed Twitter name of the account, the number of replies, retweets and likes. <br> We created a dictionary to map the usernames of the Twitter accounts to the political party. Afterwards, we defined a function that creates a dataframe with the scraped tweets from snscrape given the Twitter username and party. With the tweets scraped we only get the real tweets and reply from the politician and no retweets what helped in the analysis as we only got media content created by the desired target.

In [ ]:
twitter_name_party_dic = {'rbrinkhaus': 'CDU', 'groehe': 'CDU', 'NadineSchoen': 'CDU', 'n_roettgen': 'CDU',
                          'peteraltmaier': 'CDU', 'jensspahn': 'CDU', 'MatthiasHauer': 'CDU', 'c_lindner': 'FDP',
                          'MarcoBuschmann': 'FDP', 'starkwatzinger': 'FDP', 'Lambsdorff': 'FDP',
                          'johannesvogel': 'FDP', 'KonstantinKuhle': 'FDP', 'MAStrackZi': 'FDP',
                          'larsklingbeil': 'SPD', 'EskenSaskia': 'SPD', 'hubertus_heil': 'SPD', 'HeikoMaas': 'SPD',
                          'MartinSchulz': 'SPD', 'KarambaDiaby': 'SPD', 'Karl_Lauterbach': 'SPD',
                          'SteffiLemke': 'Grüne', 'cem_oezdemir': 'Grüne', 'GoeringEckardt': 'Grüne',
                          'KonstantinNotz': 'Grüne', 'BriHasselmann': 'Grüne', 'svenlehmann': 'Grüne',
                          'ABaerbock': 'Grüne', 'SWagenknecht': 'Linke', 'b_riexinger': 'Linke',
                          'NiemaMovassat': 'Linke', 'jankortemdb': 'Linke', 'DietmarBartsch': 'Linke',
                          'GregorGysi': 'Linke', 'SevimDagdelen': 'Linke', 'Alice_Weidel': 'AFD',
                          'Beatrix_vStorch': 'AFD', 'JoanaCotar': 'AFD', 'StBrandner': 'AFD',
                          'Tino_Chrupalla': 'AFD', 'GtzFrmming': 'AFD', 'Leif_Erik_Holm': 'AFD',
                          'ABaerbockArchiv':"Grüne"
                          }

In [ ]:
def retrieve_user_tweets(twitter_name, party):
    tweets_list = []
    search_string = "from:" + twitter_name
    for tweet in tqdm((sntwitter.TwitterSearchScraper(search_string).get_items())):
        tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.displayname,
                                tweet.replyCount, tweet.retweetCount, tweet.likeCount])
    tweets_df = pd.DataFrame(tweets_list, columns=['datetime', 'tweet_id', 'text', 'username', 'name',
                                                   'reply_count', 'retweet_count', 'like_count'])
    tweets_df["party"] = party
    return tweets_df

### 3. Scrape twitter data

We define a for loop applying our defined function and appending the generated dataframe to one big corpus for the Twitter data. After scraping the tweets for every politician, we ended up with data frames that contained date, id, text, username, Twitter name, reply count, retweet count and like count of each individual tweet. We combined those data frames to one big corpus containing all the tweets by concatenating the data frames. With this corpus of around 300000 tweets, we could start preprocessing and analyzing the tweets.

In [ ]:
twitter_name_tweets_list = []
for key in twitter_name_party_dic:
    print(key)
    twitter_name_tweets_list.append(retrieve_user_tweets(key, twitter_name_party_dic[key]))
tweets = pd.concat(twitter_name_tweets_list, ignore_index=True)
tweets.to_csv("../data/raw/tweets_scraped.csv", index = False)